# Benchmark collection models

https://github.com/LoosC/Benchmark-Models/issues

In [ ]:
import libsbml
import os
import sys
sys.path.insert(0, os.path.split(os.path.split(os.getcwd())[0])[0])
import pypesto
import sys
import amici
import importlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pypesto.objective.amici_objective import (import_sbml_model, 
                                               amici_objective_from_measurement_file)

models = ['Bachmann_MSB2011', 'beer_MolBioSystems2014', 'boehm_JProteomeRes2014',
 'Borghans_BiophysChem1997', 'Brannmark_JBC2010', 'Bruno_JExpBio2016',
'Chen_MSB2009', 'Crauste_CellSystems2017', 'Elowitz_Nature2000',
'Fiedler_BMC2016', 'Fujita_SciSignal2010', 'Hass_PONE2017',
'Isensee_JCB2018', 'Lucarelli_CellSystems_2018',
'Merkle_PCB2016', 'Raia_CancerResearch2011',
'Schwen_PONE2014','Sneyd_PNAS2002', 'Sobotta_Frontiers2017',
'Swameye_PNAS2003', 'Weber_BMC2015','Zheng_PNAS2012']

model_root = '/home/dweindl/src/Benchmark-Models/hackathon_contributions_new_data_format'

# TODO Zheng_PNAS2012
benchmark_model = 'Zheng_PNAS2012'
condition_filename = os.path.join(model_root, benchmark_model, f'experimentalCondition_{benchmark_model}.tsv')
measurement_filename = os.path.join(model_root, benchmark_model,f'measurementData_{benchmark_model}.tsv')
parameter_filename = os.path.join(model_root, benchmark_model, f'parameters_{benchmark_model}.tsv')
sbml_model_file = os.path.join(model_root, benchmark_model, f'model_{benchmark_model}_l2v4.xml')
model_name = f'model_{benchmark_model}'
model_output_dir = f'deleteme-{model_name}'

## Import model

In [ ]:
rebuild = True
if rebuild:
    import_sbml_model(sbml_model_file=sbml_model_file,
                      condition_file=condition_filename,
                      measurement_file=measurement_filename,
                      model_output_dir=model_output_dir,
                      model_name=model_name)

In [ ]:
sys.path.insert(0, os.path.abspath(model_output_dir))
model_module = importlib.import_module(model_name)

model = model_module.getModel()
model.requireSensitivitiesForAllParameters()

solver = model.getSolver()
solver.setSensitivityMethod(amici.SensitivityMethod_forward)
solver.setSensitivityOrder(amici.SensitivityOrder_first)

print("Model parameters:", list(model.getParameterIds()))
print()
print("Model const parameters:", list(model.getFixedParameterIds()))
print()
print("Model outputs:   ", list(model.getObservableIds()))
print()
print("Model states:    ", list(model.getStateIds()))

## Create objective function

In [ ]:
#from pypesto.logging import log_to_console
#log_to_console()

In [ ]:
# Create objective function instance from model and measurements
model.setParameterScale(amici.ParameterScaling_log10)
objective = amici_objective_from_measurement_file(amici_model=model, 
                                            condition_filename=condition_filename,
                                            measurement_filename=measurement_filename, 
                                            amici_solver=solver)

In [ ]:
# load nominal parameters from parameter description file
parameter_df = pd.read_csv(parameter_filename, index_col=0, sep='\t')
nominal_x = parameter_df.loc[list(model.getParameterIds())].nominalValue.values
#nominal_x = np.power(10, nominal_x)
for i, p in enumerate(model.getParameterIds()):
    if p.startswith('noise'):
        nominal_x[i] = -2.63 #np.power(10, -2.63)
print(nominal_x)

In [ ]:
# evaluate with nominal parameters
llh = objective(x=nominal_x)
print(f'llh: {llh}, lh: {np.exp(llh)}')

## Run optimization

In [ ]:
# create optimizer object which contains all information for doing the optimization
optimizer = pypesto.ScipyOptimizer()

optimizer.solver = 'bfgs'

parameter_ids = model.getParameterIds()

problem = pypesto.Problem(objective=objective,
                          lb=-5*np.ones(len(parameter_ids)), ub=5*np.ones(len(parameter_ids)),
                          x_names=parameter_ids)


# do the optimization
result = pypesto.minimize(problem=problem,
                          optimizer=optimizer,
                          n_starts=10) # 200

In [ ]:
fvals = result.optimize_result.get_for_key('fval')
fvals

In [ ]:
x = result.optimize_result.get_for_key('x')
x

## Visualization
Create waterfall and parameter plot

In [ ]:
# waterfall, parameter space,
import pypesto.visualize

pypesto.visualize.waterfall(result)
pypesto.visualize.parameters(result)

### Save data simulated with optimal parameters 

In [ ]:
# TODO: visualize goodness of fit
x = result.optimize_result.list[0]['x']
dfs = objective.simulations_to_measurement_df(x)

In [ ]:
dfs